In [1]:
from pyspark.sql import SparkSession
import pyspark

AWS_ACCESS_KEY = "minioadmin"
AWS_SECRET_KEY = "minioadmin"
AWS_S3_ENDPOINT = "http://minio_server:9000"
WAREHOUSE = "s3a://gold/" 
NESSIE_URI = "http://nessie:19120/api/v1"

conf = (
    pyspark.SparkConf()
    .setAppName("Lakehouse-Iceberg-GOLD")  
    .set('spark.jars.packages',
         'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,'
         'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,'
         'org.apache.hadoop:hadoop-aws:3.3.4,'
         'com.amazonaws:aws-java-sdk-bundle:1.12.300')
    .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
    .set("spark.sql.catalog.nessie.ref", "main")
    .set("spark.sql.catalog.nessie.authentication.type", "NONE")
    .set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .set("spark.sql.catalog.nessie.warehouse", WAREHOUSE)
    .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")
    .set("spark.sql.catalog.nessie.s3.endpoint", AWS_S3_ENDPOINT)
    .set("spark.sql.catalog.nessie.s3.access-key", AWS_ACCESS_KEY)
    .set("spark.sql.catalog.nessie.s3.secret-key", AWS_SECRET_KEY)
    .set("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .set("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .set("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")


In [2]:
df_amazon = spark.table("nessie.amazon_purchase")
df_survey = spark.table("nessie.survey")
df_fields = spark.table("nessie.fields")

### Tạo dim_customer

In [3]:
from pyspark.sql import functions as F

dim_customer = (
    df_survey.selectExpr(
        "`Survey ResponseID` as customer_id",
        "`Q-demos-age` as age_group",
        "`Q-demos-hispanic` as hispanic",
        "`Q-demos-race` as race",
        "`Q-demos-education` as education",
        "`Q-demos-income` as income",
        "`Q-demos-gender` as gender",
        "`Q-sexual-orientation` as sexual_orientation",
        "`Q-demos-state` as state",
        "`Q-amazon-use-howmany` as amazon_use_howmany",
        "`Q-amazon-use-hh-size` as amazon_use_hh_size",
        "`Q-amazon-use-how-oft` as amazon_use_how_oft",
        "`Q-substance-use-cigarettes` as substance_use_cigarettes",
        "`Q-substance-use-marijuana` as substance_use_marijuana",
        "`Q-substance-use-alcohol` as substance_use_alcohol",
        "`Q-personal-diabetes` as personal_diabetes",
        "`Q-personal-wheelchair` as personal_wheelchair",
        "`Q-life-changes` as life_changes",
        "`Q-sell-YOUR-data` as sell_your_data",
        "`Q-sell-consumer-data` as sell_consumer_data",
        "`Q-small-biz-use` as small_biz_use",
        "`Q-census-use` as census_use",
        "`Q-research-society` as research_society"
    )
    .dropDuplicates(["customer_id"])
)


In [4]:
dim_customer.limit(2).toPandas()

,customer_id,age_group,hispanic,race,education,income,gender,sexual_orientation,state,amazon_use_howmany,...,substance_use_marijuana,substance_use_alcohol,personal_diabetes,personal_wheelchair,life_changes,sell_your_data,sell_consumer_data,small_biz_use,census_use,research_society
0,R_01vNIayewjIIKMF,35 - 44 years,Yes,Black or African American,Bachelor's degree,"$25,000 - $49,999",Male,heterosexual (straight),New Jersey,1 (just me!),...,No,No,No,No,No,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes
1,R_037XK72IZBJyF69,55 - 64 years,No,White or Caucasian,Bachelor's degree,"$25,000 - $49,999",Female,heterosexual (straight),Pennsylvania,1 (just me!),...,No,Yes,No,No,No,Yes if I get part of the profit,Yes if consumers get part of the profit,I don't know,No,Yes


In [5]:
dim_customer.count()


5027

### Tạo dim_product

In [6]:
df_amazon.limit(7).toPandas()

,Order Date,Purchase Price Per Unit,Quantity,Shipping Address State,Title,ASIN/ISBN (Product Code),Category,Survey ResponseID
0,2018-12-04,7.98,1.0,NJ,SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...,B0143RTB1E,FLASH_MEMORY,R_01vNIayewjIIKMF
1,2018-12-22,13.99,1.0,NJ,Betron BS10 Earphones Wired Headphones in Ear ...,B01MA1MJ6H,HEADPHONES,R_01vNIayewjIIKMF
2,2018-12-25,10.45,1.0,NJ,Perfecto Stainless Steel Shaving Bowl. Durable...,B06XWF9HML,DISHWARE_BOWL,R_01vNIayewjIIKMF
3,2018-12-25,10.00,1.0,NJ,Proraso Shaving Cream for Men,B00837ZOI0,SHAVING_AGENT,R_01vNIayewjIIKMF
4,2019-02-18,10.99,1.0,NJ,Micro USB Cable Android Charger - Syncwire [2-...,B01GFB2E9M,COMPUTER_PROCESSOR,R_01vNIayewjIIKMF
5,2019-02-18,4.99,1.0,NJ,Amazon Basics USB 2.0 Charger Cable - A-Male t...,B00NH13S44,COMPUTER_ADD_ON,R_01vNIayewjIIKMF
6,2019-03-15,124.99,1.0,NJ,"Fire HD 8 Tablet (8"" HD Display, 32 GB, withou...",B077H6L7T9,AMAZON_TABLET,R_01vNIayewjIIKMF


In [8]:
import pandas as pd

# Tăng giới hạn hiển thị cho chuỗi dài
pd.set_option('display.max_colwidth', None)

# Chuyển 2 dòng đầu tiên sang pandas
df_amazon_pd = df_amazon.select("ASIN/ISBN (Product Code)").limit(50).toPandas()

# In ra toàn bộ nội dung cột Title
print(df_amazon_pd)

   ASIN/ISBN (Product Code)
0                B0143RTB1E
1                B01MA1MJ6H
2                B06XWF9HML
3                B00837ZOI0
4                B01GFB2E9M
5                B00NH13S44
6                B077H6L7T9
7                B07L84ZZXC
8                B07CG71KQ1
9                B079GFF4HZ
10               B01M0Q84BR
11               B01MA1MJ6H
12               B073JYC4XM
13               B00I7DT454
14               B01MA1MJ6H
15               B0043T7FXE
16               B01M2V05RE
17               B07BCZMBNR
18               B075CMYT43
19               B01MZ956SX
20               B07JCJP1DK
21               B07JD5B8SP
22               B071XGLB1S
23               B071ZBV7DY
24               B07D5V2ZXD
25               B01BNEWDFQ
26               B01NBYY28W
27               B019FGCOW6
28               B072JYDQ7N
29               B00MUJU33S
30               B01IQ00B04
31               B017ALC6GW
32               B0741FV7ZV
33               B006MIPW70
34               B01

In [17]:
from pyspark.sql.functions import col

dim_product = df_amazon.select(
    col("ASIN/ISBN (Product Code)").alias("product_id"),
    col("Title").alias("product_title"),
    col("Category").alias("product_category")
).dropDuplicates(["product_id"])

In [20]:
dim_product.limit(10).toPandas()

,product_id,product_title,product_category
0,000217653X,THE DINAH'S CUPBOARD COOK BOOK: Recipes and Menus for Elegant Home Entertaining,ABIS_BOOK
1,0007137508,Wellington: The Iron Duke,ABIS_BOOK
2,0007302622,Duck in the Truck,ABIS_BOOK
3,000745287X,"Sharpe's Regiment: Richard Sharpe and the Invasion of France, June to November 1913. Bernard Cornwell",ABIS_BOOK
4,0007483791,Deep Time,ABIS_BOOK
5,0007510837,Collins German Dictionary Complete and Unabridged Edition (Collins Complete and Unabridged),ABIS_BOOK
6,0007544790,My Virgin Kitchen: Delicious recipes you can make every day,ABIS_BOOK
7,000756032X,Born into the Children of God: My life in a religious sex cult and my struggle for survival on the outside,ABIS_BOOK
8,0008100713,Well Gardened Mind,ABIS_BOOK
9,000813622X,Frog & Toad The Complete Collection,ABIS_BOOK


In [19]:
dim_product.count()


857821

### Tạo dim_time

In [26]:
from pyspark.sql.functions import year, month, dayofmonth, quarter, dayofweek, monotonically_increasing_id, date_format

dim_time = df_amazon.select(col("Order Date").alias("order_date")).dropDuplicates()

dim_time = dim_time.withColumn("year", year(col("order_date"))) \
                   .withColumn("month", month(col("order_date"))) \
                   .withColumn("day", dayofmonth(col("order_date"))) \
                   .withColumn("quarter", quarter(col("order_date"))) \
                   .withColumn("weekday", dayofweek(col("order_date"))) \
                   .withColumn("weekday_name", date_format(col("order_date"), "EEEE")) \
                   .withColumn("time_id", monotonically_increasing_id())

dim_time.show()


+----------+----+-----+---+-------+-------+------------+-------+
|order_date|year|month|day|quarter|weekday|weekday_name|time_id|
+----------+----+-----+---+-------+-------+------------+-------+
|2021-01-27|2021|    1| 27|      1|      4|   Wednesday|      0|
|2021-06-22|2021|    6| 22|      2|      3|     Tuesday|      1|
|2022-07-31|2022|    7| 31|      3|      1|      Sunday|      2|
|2019-06-04|2019|    6|  4|      2|      3|     Tuesday|      3|
|2021-10-11|2021|   10| 11|      4|      2|      Monday|      4|
|2021-12-18|2021|   12| 18|      4|      7|    Saturday|      5|
|2022-03-28|2022|    3| 28|      1|      2|      Monday|      6|
|2021-11-13|2021|   11| 13|      4|      7|    Saturday|      7|
|2018-08-10|2018|    8| 10|      3|      6|      Friday|      8|
|2021-08-27|2021|    8| 27|      3|      6|      Friday|      9|
|2020-08-24|2020|    8| 24|      3|      2|      Monday|     10|
|2019-05-08|2019|    5|  8|      2|      4|   Wednesday|     11|
|2018-05-28|2018|    5| 2

### Tạo dim_location

In [23]:
df_amazon.select("Shipping Address State").distinct().show(truncate=False)


+----------------------+
|Shipping Address State|
+----------------------+
|AZ                    |
|SC                    |
|LA                    |
|MN                    |
|NJ                    |
|DC                    |
|OR                    |
|VA                    |
|RI                    |
|WY                    |
|KY                    |
|NH                    |
|MI                    |
|NV                    |
|WI                    |
|ID                    |
|CA                    |
|NE                    |
|CT                    |
|MT                    |
+----------------------+
only showing top 20 rows



In [27]:
from pyspark.sql.functions import col, monotonically_increasing_id, udf
from pyspark.sql.types import StringType

# Bảng tra cứu bang → tên bang và vùng miền
state_lookup = {
    "NJ": ("New Jersey", "Northeast"),
    "NY": ("New York", "Northeast"),
    "CA": ("California", "West"),
    "TX": ("Texas", "South"),
    "AZ": ("Arizona", "West"),
    "SC": ("South Carolina", "South"),
    "LA": ("Louisiana", "South"),
    "MN": ("Minnesota", "Midwest"),
    "DC": ("District of Columbia", "South"),
    "OR": ("Oregon", "West"),
    "VA": ("Virginia", "South"),
    "RI": ("Rhode Island", "Northeast"),
    "WY": ("Wyoming", "West"),
    "KY": ("Kentucky", "South"),
    "NH": ("New Hampshire", "Northeast"),
    "MI": ("Michigan", "Midwest"),
    "NV": ("Nevada", "West"),
    "WI": ("Wisconsin", "Midwest"),
    "ID": ("Idaho", "West"),
    "NE": ("Nebraska", "Midwest"),
    "CT": ("Connecticut", "Northeast"),
    "MT": ("Montana", "West")
}

# UDF để lấy tên bang
def get_state_name(code):
    return state_lookup.get(code, ("Unknown", "Unknown"))[0]

# UDF để lấy vùng miền
def get_region(code):
    return state_lookup.get(code, ("Unknown", "Unknown"))[1]

udf_state_name = udf(get_state_name, StringType())
udf_region = udf(get_region, StringType())

# Tạo dim_location
dim_location = df_amazon.select(col("Shipping Address State").alias("state_code")).dropDuplicates()

dim_location = dim_location.withColumn("location_id", monotonically_increasing_id()) \
                           .withColumn("state_name", udf_state_name(col("state_code"))) \
                           .withColumn("region", udf_region(col("state_code")))

dim_location.show(truncate=False)


+----------+-----------+--------------------+---------+
|state_code|location_id|state_name          |region   |
+----------+-----------+--------------------+---------+
|AZ        |0          |Arizona             |West     |
|SC        |1          |South Carolina      |South    |
|LA        |2          |Louisiana           |South    |
|MN        |3          |Minnesota           |Midwest  |
|NJ        |4          |New Jersey          |Northeast|
|DC        |5          |District of Columbia|South    |
|OR        |6          |Oregon              |West     |
|VA        |7          |Virginia            |South    |
|RI        |8          |Rhode Island        |Northeast|
|WY        |9          |Wyoming             |West     |
|KY        |10         |Kentucky            |South    |
|NH        |11         |New Hampshire       |Northeast|
|MI        |12         |Michigan            |Midwest  |
|NV        |13         |Nevada              |West     |
|WI        |14         |Wisconsin           |Mid

### Tạo fact_order

In [28]:
from pyspark.sql.functions import col, expr

# Join dim_time để lấy time_id
fact_order = df_amazon \
    .join(dim_time, df_amazon["Order Date"] == dim_time["order_date"], "left") \
    .join(dim_product, df_amazon["ASIN/ISBN (Product Code)"] == dim_product["product_id"], "left") \
    .join(dim_customer, df_amazon["Survey ResponseID"] == dim_customer["customer_id"], "left") \
    .join(dim_location, df_amazon["Shipping Address State"] == dim_location["state_code"], "left")

# Chọn các cột cần thiết cho fact_order
fact_order = fact_order.select(
    col("time_id"),
    col("customer_id"),
    col("product_id"),
    col("location_id"),
    col("Purchase Price Per Unit").alias("purchase_price_per_unit"),
    col("Quantity").alias("quantity"),
    (col("Purchase Price Per Unit") * col("Quantity")).alias("total_price")
)

fact_order.show(5, truncate=False)


+-------+-----------------+----------+-----------+-----------------------+--------+-----------+
|time_id|customer_id      |product_id|location_id|purchase_price_per_unit|quantity|total_price|
+-------+-----------------+----------+-----------+-----------------------+--------+-----------+
|1110   |R_01vNIayewjIIKMF|B01MA1MJ6H|4          |13.99                  |1.0     |13.99      |
|138    |R_01vNIayewjIIKMF|B06XWF9HML|4          |10.45                  |1.0     |10.45      |
|138    |R_01vNIayewjIIKMF|B00837ZOI0|4          |10.0                   |1.0     |10.0       |
|1089   |R_01vNIayewjIIKMF|B01GFB2E9M|4          |10.99                  |1.0     |10.99      |
|1089   |R_01vNIayewjIIKMF|B00NH13S44|4          |4.99                   |1.0     |4.99       |
+-------+-----------------+----------+-----------+-----------------------+--------+-----------+
only showing top 5 rows



In [31]:
fact_order_full = (
    fact_order
    .join(dim_time, on="time_id", how="left")
    .join(dim_customer, on="customer_id", how="left")
    .join(dim_product, on="product_id", how="left")
    .join(dim_location, on="location_id", how="left")
)

fact_order_full.limit(10).toPandas()


,location_id,product_id,customer_id,time_id,purchase_price_per_unit,quantity,total_price,order_date,year,month,...,sell_your_data,sell_consumer_data,small_biz_use,census_use,research_society,product_title,product_category,state_code,state_name,region
0,4,B01MA1MJ6H,R_01vNIayewjIIKMF,1110,13.99,1.0,13.99,2018-12-22,2018,12,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,"Betron BS10 Earphones Wired Headphones in Ear Noise Isolating Earbuds with Microphone and Volume Control Powerful Bass Driven Sound, 12mm Large Drivers, Ergonomic Design",HEADPHONES,NJ,New Jersey,Northeast
1,4,B06XWF9HML,R_01vNIayewjIIKMF,138,10.45,1.0,10.45,2018-12-25,2018,12,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,Perfecto Stainless Steel Shaving Bowl. Durable Metal Mug For Shaving Soap & Cream. Perfect Addition To Your Wet Shaving Kit. Double Layer Smooth Shave Unbreakable Mug With Heat Insulation,DISHWARE_BOWL,NJ,New Jersey,Northeast
2,4,B077H6L7T9,R_01vNIayewjIIKMF,1804,124.99,1.0,124.99,2019-03-15,2019,3,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,"Fire HD 8 Tablet (8"" HD Display, 32 GB, without Special Offers) - Black (Previous Generation - 8th)",AMAZON_TABLET,NJ,New Jersey,Northeast
3,4,B00837ZOI0,R_01vNIayewjIIKMF,138,10.00,1.0,10.00,2018-12-25,2018,12,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,Proraso Shaving Cream for Men,SHAVING_AGENT,NJ,New Jersey,Northeast
4,4,B01GFB2E9M,R_01vNIayewjIIKMF,1089,10.99,1.0,10.99,2019-02-18,2019,2,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,"Micro USB Cable Android Charger - Syncwire [2-Pack 6.6ft] Super-Durable Nylon Braided Fast Sync&Charging Cord for Samsung Galaxy S7 Edge/S7/S6, HTC, LG, Sony, Xbox One, PS4 - Space Grey",COMPUTER_PROCESSOR,NJ,New Jersey,Northeast
5,4,B00NH13S44,R_01vNIayewjIIKMF,1089,4.99,1.0,4.99,2019-02-18,2019,2,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,Amazon Basics USB 2.0 Charger Cable - A-Male to Mini-B Cord - 3 Feet (0.9 Meters),ELECTRONIC_CABLE,NJ,New Jersey,Northeast
6,4,B0143RTB1E,R_01vNIayewjIIKMF,246,7.98,1.0,7.98,2018-12-04,2018,12,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory Card up to 80MB/s (SDSDUNC-016G-GN6IN),FLASH_MEMORY,NJ,New Jersey,Northeast
7,4,B07CG71KQ1,R_01vNIayewjIIKMF,414,9.99,1.0,9.99,2019-05-02,2019,5,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,UGREEN Tablet Stand Holder Adjustable Portable Desktop Holder Dock Compatible for iPad 10.2 iPad Pro 11 Inch iPad 9.7 iPad Mini 5 4 3 2 iPad Air iPhone 13 12 Pro Max 11 XS XR Black,PORTABLE_ELECTRONIC_DEVICE_STAND,NJ,New Jersey,Northeast
8,4,B07L84ZZXC,R_01vNIayewjIIKMF,890,12.99,1.0,12.99,2019-04-23,2019,4,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,"Men's Leather Belt, Ratchet Dress Belt with Automatic Buckle for Father's Day Gift by WERFORU",APPAREL_BELT,NJ,New Jersey,Northeast
9,4,B079GFF4HZ,R_01vNIayewjIIKMF,414,12.79,1.0,12.79,2019-05-02,2019,5,...,Yes if I get part of the profit,Yes if consumers get part of the profit,No,No,Yes,"Betron B25 in-Ear Headphones Earphones with Microphone and Volume Controller, Noise Isolating Earbud Tips, 3.5mm Head Phone Jack",HEADPHONES,NJ,New Jersey,Northeast


In [32]:
# Lưu fact_order vào Iceberg Gold Layer
fact_order.writeTo("nessie.fact_order").createOrReplace()

# Lưu dim_customer
dim_customer.writeTo("nessie.dim_customer").createOrReplace()

# Lưu dim_product
dim_product.writeTo("nessie.dim_product").createOrReplace()

# Lưu dim_time
dim_time.writeTo("nessie.dim_time").createOrReplace()

# Lưu dim_location
dim_location.writeTo("nessie.dim_location").createOrReplace()


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55822)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [33]:
# Lưu fact_order vào Iceberg Gold Layer
fact_order.writeTo("nessie.fact_order_full").createOrReplace()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_com

Py4JError: An error occurred while calling o337.writeTo

In [34]:
spark.stop()  # Dừng SparkSession hiện tại

ConnectionRefusedError: [Errno 111] Connection refused